# تبدیل فایل سرفصل درسها به صورت مارک داون مناسب

## M. Amintoosi

In [36]:
# !pip install markdown-strings


In [37]:
import io
import re
import markdown_strings
import pandas as pd


In [38]:
def extract_elements(input_str):
    start_idx = input_str.find('|',1)
    start_idx = input_str.find('|',start_idx) # Move to next |
    end_idx = input_str.find('|',start_idx+1)
    prreq = input_str[start_idx+1:end_idx]
    prreq = re.sub('\*', '', prreq)
    prreq = prreq.strip()
    if prreq == '':
        prreq = 'ندارد'
    # print(prreq)

    # نوع درس
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    c_cat = input_str[start_idx+1:end_idx]
    c_cat = re.sub('\*', '', c_cat)
    if 'n' in c_cat  or '¢' in c_cat:
        c_cat = c_cat.replace('n','').strip()
        c_cat = c_cat.replace('¢','').strip()
    elif 'حل تمرین' in c_cat:
        c_cat = re.sub('حل تمرین','',c_cat)
    else:
        c_cat = ''
    # print(c_cat)

    # نوع واحد
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    unit_type = input_str[start_idx+1:end_idx]
    unit_type = re.sub('\*', '', unit_type)
    if 'n' in unit_type or '¢' in unit_type:
        unit_type = unit_type.replace('n','').strip()
        unit_type = unit_type.replace('¢','').strip()
    else:
        unit_type = ''
    # print(unit_type)
    return prreq, c_cat, unit_type

In [39]:
# Load the file into file_content
file_content = [line for line in open('input/Computer_Science_15_md.md')]
col_names = ['fa_title','en_title','en_file_name','c_cat','complementary','prreq','coreq',\
                           'c_unit','unit_type','c_hours','c_ex','c_goal1','c_goal2','sar_fasl','c_refs']
df = pd.DataFrame(columns=col_names)

fa_col_names = ['عنوان درس (فارسی):', 'عنوان درس (انگلیسی):', 'نوع درس:', 'آموزش تکمیلی:','پیش‌نیاز:','هم‌نیاز',\
                 'تعداد واحد:', 'نوع واحد:', 'تعداد ساعت:','حل تمرین:', 'هدف کلی:', 'اهداف ویژه:',\
                      'سر فصل یا رئوس مطالب','فهرست منابع پیشنهادی']
# df.set_index('fa_title',inplace = True)
line_no = 0
while line_no < len(file_content):
    line = file_content[line_no]
    pattern = r"\| --- \|"
    result = re.sub(pattern, "", line)
    # pattern = r"\|"
    # result = re.sub(pattern, "", result)
    srch_str = "عنوان درس به فارسی:"
    # print(result.find(srch_str))
    if result.find(srch_str) != -1:
        result = re.sub("<a.*?/a>",'',result)
        start_idx = result.find('<h3>') + 4
        end_idx = result.rfind('</h3>')
        fa_title = result[start_idx:end_idx]
        fa_title = re.sub('\*', '', fa_title)
        fa_title = fa_title.strip()
        line_no += 1
        print(fa_title)
        continue

    srch_str = "عنوان درس به انگلیسی:"
    # print(result , result.find(srch_str))
    if result.find(srch_str) != -1:
        start_idx = result.find('|',1)
        end_idx = result.find('|',start_idx+1)
        en_title = result[start_idx+1:end_idx]
        en_title = re.sub('\*', '', en_title)
        en_title = en_title.strip()
        en_file_name = re.sub('\&', 'and', en_title)
        en_file_name = re.sub(' ', '-', en_file_name)
        print(en_title)

        # پیش نیاز
        line_no += 1
        line = file_content[line_no]
        prreq, c_cat, unit_type = extract_elements(line)
        print(prreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        coreq, c_cat_, unit_type_ = extract_elements(line)
        # if c_cat_ != '':
        #     print('\n', c_cat_, '\n')
        c_cat, unit_type = c_cat+c_cat_,  unit_type+unit_type_
        print(coreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        c_unit, c_ex, c_cat_ = extract_elements(line)
        if unit_type == '':
            unit_type = "نظری-عملی"
        c_cat = c_cat+c_cat_
        print(c_unit, c_ex, c_cat)

        line_no += 1
        line = file_content[line_no]
        c_hours, _, _ = extract_elements(line)
        print(c_hours, '\n')
       
        while True:
            line_no += 1
            if line_no >= len(file_content):
                break
            line = file_content[line_no]
            if line.find(fa_col_names[-2]) != -1: #سرفصل
                break
        if line_no >= len(file_content):
                break

        # continue
        # خواندن سرفصل
        sar_fasl =[]
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find(fa_col_names[-1]) == -1: #refs
            if line.strip() != '':
                sar_fasl.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]

        if line_no >= len(file_content):
            break    

        refs = []
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find("عنوان درس به فارسی:") == -1:
            if line.strip() != '':
                refs.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]
            # print(line_no)

        if line.find("عنوان درس به فارسی:") != -1:
            line_no -= 1
        this_course ={'fa_title':fa_title, 'en_title':en_title, 'en_file_name':en_file_name,'c_cat':c_cat,'prreq':prreq,'coreq':coreq,\
                           'c_unit':c_unit,'unit_type':unit_type,'c_hours':c_hours,'c_ex':c_ex,\
                            'sar_fasl':sar_fasl,'c_refs':refs}
        df = df.append(this_course,ignore_index=True)
    line_no += 1
    # print('')
    
df

ریاضی عمومی 1
Calculus I
ندارد پایه نظری
ندارد پایه نظری
4  دارد پایه
48 

ریاضی عمومی 2
Calculus II
ریاضی عمومی 1 پایه نظری
ندارد پایه نظری
4  دارد پایه
64 

معادلات دیفرانسیل
Differential Equations
ریاضی عمومی ۱ پایه نظری
ندارد پایه نظری
3  دارد پایه
48 

مبانی برنامه‌نویسی
Fundamentals of Computer Programming
ندارد پایه نظری
ندارد پایه نظری
3  دارد پایه
48 

مبانی اقتصاد
Basics of Economics
ندارد پایه نظری
ندارد پایه نظری
2  ندارد پایه
32 

کارگاه کامپیوتر ۱
Computer Workshop I
ندارد پایه 
مبانی برنامه‌نویسی پایه عملی
1  ندارد پایه
32 

مبانی علوم ریاضی
Foundation of Mathematics
ندارد  نظری
ندارد تخصصی الزامی نظری
3  دارد تخصصی الزامی
48 

آمار و احتمال 1
Probability and Statistics I
ندارد  نظری
ندارد تخصصی الزامی نظری
3  دارد تخصصی الزامی
48 

تحلیل داده‌ها با نرم‌افزارهای عمومی
Graphical and Exploratory data analysis
ندارد پایه 
ندارد پایه 
1+1  پایه
16+32 

احتمال م‍قدماتی
Elementary Probability
ندارد پایه نظری
ندارد پایه نظری
1  دارد پایه
16 

برنامه‌نویسی پیشرفته
Advanced Progr

,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
0,ریاضی عمومی 1,Calculus I,Calculus-I,پایه,NaN,ندارد,ندارد,4,نظری,48,دارد,NaN,NaN,[مختصات دکارتی، مختصات قطبی، اعداد مختلط، جمع ...,[کتاب حساب دیفرانسیل و انتگرال تالیف جیمز استو...
1,ریاضی عمومی 2,Calculus II,Calculus-II,پایه,NaN,ریاضی عمومی 1,ندارد,4,نظری,64,دارد,NaN,NaN,[سیستم مختصات قطبی، نمودارهاي قطبی دنباله، سري...,[کتاب حساب دیفرانسیل و انتگرال تالیف جیمز استو...
2,معادلات دیفرانسیل,Differential Equations,Differential-Equations,پایه,NaN,ریاضی عمومی ۱,ندارد,3,نظری,48,دارد,NaN,NaN,[مفاهیم و تعاریف اولیه .معادلات جداپذیر و معاد...,"[William E. Boyce, Richard C. DiPrima, Douglas..."
3,مبانی برنامه‌نویسی,Fundamentals of Computer Programming,Fundamentals-of-Computer-Programming,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[مقدمه ای بر الگوریتم. متغیرها. ورودی خروجی. س...,"[<https://github.com/ambadidi/Python-Books>\n,..."
4,مبانی اقتصاد,Basics of Economics,Basics-of-Economics,پایه,NaN,ندارد,ندارد,2,نظری,32,ندارد,NaN,NaN,[اقتصاد: مبانی و مدل ها. معاملات، مزیت مقایسه ...,"[<a name=""_hlk932247371""></a>O’Sullivan, A., S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,مبانی رایانش ابری,Cloud Computing Fundamentals,Cloud-Computing-Fundamentals,تخصصی اختیاری,NaN,شبکه های کامپيوتری ،اصول سيستم های عامل,ندارد,۳,نظری,48,دارد,NaN,NaN,[1- آشنایی با مفاهیم رایانش ابری، مدلهای مختلف...,"[1. ` `K. Chandrasekaran, *Essentials of c..."
80,رایانش چند‌هسته‌ای,Multicore Computing,Multicore-Computing,تخصصی اختیاری,NaN,برنامه‌نویسی پیشرفته، اصول سیستم‌های کامپیوتری,ندارد,۳,نظری,48,دارد,NaN,NaN,[معرفی معماری سیستم‌های چندهسته‌ای و مدل‌های ب...,[1. *D. A. Patterson and J. L. Hennessy. Compu...
81,سیگنال‌ها و سیستم‌ها,Signals and Systems,Signals-and-Systems,تخصصی اختیاری,NaN,معادلات دیفرانسيل,ندارد,۳,نظری,48,دارد,NaN,NaN,[--مفاهیم اولیه، سيگنال های زمان پيوسته و گسست...,"[A.V. Oppenheim, A.S. Willsky, S.H. Nawab, *Si..."
82,مدیریت پروژه‌های فناوری اطلاعات,Information Technology Project Management,Information-Technology-Project-Management,تخصصی اختیاری,NaN,ندارد,ندارد,۳,نظری,48,ندارد,NaN,NaN,[مقدمات، اهداف، برنامه‌ی درسی و توافق بر الگوی...,[Jack T. Marchewka. Information Technology Pro...


In [40]:
df.to_excel('output/lessons.xlsx')

ایجاد فایل برای تک تک درسها

In [41]:
for index, row in df.iterrows():
    # print(row['fa_title'], row['en_title'])
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
    # print(r_path)
    lesson_name_en = row['en_title'].strip()
    lesson_name_en = re.sub('\&', 'and', lesson_name_en)
    lesson_name_en = re.sub(' ', '-', lesson_name_en)
    lesson_file_name = r_path+'/'+lesson_name_en+'.md'
    # print(lesson_file_name)
    lesson_writer = open(lesson_file_name, 'w')
    md_str = markdown_strings.header(row['fa_title']+'\n', 1)
    lesson_writer.write(md_str)
    md_str = markdown_strings.header(row['en_title']+'\n', 2)
    lesson_writer.write(md_str)
    lesson_writer.write(markdown_strings.horizontal_rule())
    lesson_writer.write('\n')
    lesson_writer.write(table_from_rows([\
        ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"],\
        ["پیش‌نیاز:",row['prreq'],"گروه درس:",row['c_cat']],\
        ["هم‌نیاز:",row['coreq'],"نوع درس:",row['unit_type']],\
        ["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']],\
        ["حل تمرین:",row['c_ex'],"",""]\
        ]))
    lesson_writer.write('\n')

    lesson_writer.close()
    if index > 15:
        break

ایجاد فایل yml

In [42]:
df_sorted = df.sort_values(by=['c_cat'], ascending=False)
# df_sorted

In [45]:
yml_file_name = '../mkdocs.yml'
yml_writer = open(yml_file_name, 'w')
yml_str = "site_name: علوم کامپیوتر فردوسی مشهد\n\
nav:\n\
  - صفحه اصلی: index.md\n\
  - درباره: about.md\n\
  - دروس پایه:\n\
"
yml_writer.write(yml_str)
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        yml_str = "  - دروس الزامی:"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        yml_str = "  - دروس اختیاری:"

    if curr_cat != prev_cat:
        yml_writer.write(yml_str)
        yml_writer.write('\n')
    prev_cat = curr_cat
    # print(r_path)
    lesson_file_name = r_path+'/'+row['en_file_name']+'.md'
    # print(lesson_file_name)
    yml_str = '    - "'+row['fa_title']+'": '+lesson_file_name+'\n'
    yml_writer.write(yml_str)

    # if index > 80:
    #     break

yml_writer.close()
